# Spark: Duplicate Reviews

In [18]:
import random
import numpy as np
import math
import csv
import json
import pyspark
import hashlib
import sys

from pyspark import SparkConf, AccumulatorParam
from pyspark.context import SparkContext
from pyspark import StorageLevel
from pyspark.sql import SparkSession

path = 'a2_p2_Mittal_114855990_OUTPUT.txt'
sys.stdout = open(path, 'w')

session = SparkSession.builder\
            .appName("Assignment2")\
            .config("spark.driver.memory", "15g") \
            .config("spark.dynamicAllocation.enabled", "true") \
            .config("spark.shuffle.service.enabled", "true") \
            .config("spark.executor.cores", "4") \
            .config("spark.rpc.message.maxSize", "256") \
            .config("spark.executor.memory", "14g") \
            .config("spark.executor.instances", "4") \
            .getOrCreate()
    
sc = session.sparkContext
# sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))

class SetAccum(AccumulatorParam):
    def zero(self, zeroValue = set()):#overwrite this
        return set()
    def addInPlace(self, v1, v2):#overwrite this
        v1.update(v2)
        return v1
SetAcc = sc.accumulator(set(), SetAccum())

seeds = []
for i in range(1000):
    random.seed(i)
    a = random.randint(0, 10000)
    b = random.randint(0, 10000)
    l = [a, b]
    seeds.append(l)

seedListSC = sc.broadcast(seeds)

## Taken fron CHATGPT using PROMPT "python code to create k-shingles of string"
def create_k_shingles(string, k):
    """
    Creates k-shingles of a string.
    
    Parameters:
    string (str): The input string.
    k (int): The length of the shingles.
    
    Returns:
    list: A list of k-shingles of the input string.
    """
    shingles = set()
    n = len(string)
    p = n - k + 1
    for i in range(0,p):
        shingles.add(string[i:i+k])
    SetAcc.add(shingles)
    return shingles

# dataRDD = sc.textFile('hdfs:/data/Appliances_5.json')
filename = '../Project/Social-Listening-Dev/filtered_data/mental_health_sample.txt'
dataRDD = sc.textFile(filename)
dataRDD = dataRDD.map(lambda x: x.split(','))

Drop duplicate posts.

In [19]:
dataRDD = dataRDD.map(lambda x: (x[1], x[0])).reduceByKey(lambda x, y: x).map(lambda x: (x[1], x[0]))

Assign unique IDs to each post.

In [20]:
dataRDD = dataRDD.zipWithUniqueId().map(lambda x: (x[1], x[0][0], x[0][1]))

Create shingles.

"""
We need to update k for shingle size.
"""

In [21]:
shingledRDD = dataRDD.map(lambda x: (x[0], x[1], x[2], create_k_shingles(x[2], 4)))
shingledRDD.persist(StorageLevel.MEMORY_AND_DISK)
shingledRDD.count()

1001

Get number of unique shingles and convert to a broadcasted variable.

In [22]:
set_size = len(SetAcc.value)
# print(len(SetAcc.value))
hash_rangeSC = sc.broadcast(set_size)
hashShingles = dict()
for k, i in enumerate(SetAcc.value):
    hf = k%len(seedListSC.value)
    hashShingles[i] = (hf, (seedListSC.value[hf][0] * hash(i) + seedListSC.value[hf][1])%1001)
    
hashShinglesSC = sc.broadcast(hashShingles)
num_bucketsSC = sc.broadcast(100)

In [23]:
class DictAccum(AccumulatorParam):
    def zero(self, zeroValue = dict()):#overwrite this
        return dict()
    def addInPlace(self, v1, v2):#overwrite this
        for i in v2.keys():
            v1[i] = v2[i]
        return v1
DictAcc = sc.accumulator(dict(), DictAccum())

In [24]:
class ListAccum(AccumulatorParam):
    def zero(self, zeroValue = []):#overwrite this
        return []
    def addInPlace(self, v1, v2):#overwrite this
        return v1+v2

listAcc = sc.accumulator([], ListAccum())

In [25]:
rows = sc.broadcast(10)
bands = sc.broadcast(100)

In [26]:
def listRemoveInf(l):
    ans  = []
    for i, v in enumerate(l):
        if v!= float('inf'):
            ans.append(str(i)+"$"+str(v))
    return ans

In [27]:
def AddToBuckets(key, signatures):
    allSetList = []
    t = [hash('|'.join(listRemoveInf(signatures[i:i+rows.value])))%num_bucketsSC.value for i in range(0, 1000, rows.value)]
    l = 0
    for r in t:
        if r == "":
            continue
        allSetList.append(str(l)+"_"+str(r)+"_"+str(key))
        l+=rows.value
    listAcc.add(allSetList)
    return

In [28]:
def minhash_signature_filtered(key, shingles):
    try:
        signature = [float('inf')] * len(seedListSC.value)
        # hash_range = len(shingles)
        for shingle in shingles:
            hf = hashShinglesSC.value[shingle][0]
            value = hashShinglesSC.value[shingle][1]
            signature[hf] = min(signature[hf], value)
        d = dict()
        d[key] = shingles
        DictAcc.add(d)
        # print((key,signature))
        AddToBuckets(key, signature)
    except:
        return None, None, None
    return (key,signature,shingles)

In [29]:
filterList = shingledRDD.map(lambda x: x[0]).take(5)
filterListSC = sc.broadcast(filterList)
filterRDD = shingledRDD.filter(lambda x: x[0] in filterListSC.value).map(lambda x: (minhash_signature_filtered(x[0], x[3])))
filterRDD.persist(StorageLevel.MEMORY_AND_DISK)

PythonRDD[22] at RDD at PythonRDD.scala:53

In [30]:
filterDict = sc.broadcast(DictAcc.value)
selectedHashesSC = sc.broadcast(listAcc.value)

In [31]:
def createSignatureFromSet_Unfiltered(key, shingles):
    signature = [float('inf')] * len(seedListSC.value)
    for shingle in shingles:
        hf = hashShinglesSC.value[shingle][0]
        value = hashShinglesSC.value[shingle][1]
        signature[hf] = min(signature[hf], value)
    return (key, signature, shingles)

def calculateJaccard(k1,k2,m1,m2):
    i = m1.intersection(m2)
    u = m1.union(m2)
    return (k1, float(len(i)/len(u)))

def checkForCandidates(key, signatures, shingles):
    t = [hash('|'.join(listRemoveInf(signatures[i:i+rows.value])))%num_bucketsSC.value for i in range(0, 1000, rows.value)]
    l = 0
    d = dict()
    ans = []
    for k in filterListSC.value:
        d[k] = 0
    for r in t:
        if r == "":
            continue
        for k in filterListSC.value:
            if d[k]==0:
                if str(l)+"_"+str(r)+"_"+str(k) in selectedHashesSC.value:
                    d[k]+=1
            else:
                continue   
        l+=10
    for k in filterListSC.value:
        if d[k]>0:
            h = calculateJaccard(k, key, filterDict.value[k], shingles)
            if h[1]>=float(4/5):
                ans.append(h)
    return (key, ans)

In [32]:
nonFilteredRDD = shingledRDD.filter(lambda x: x[0] not in filterListSC.value).map(lambda x: createSignatureFromSet_Unfiltered(x[0], x[3]))
selectedRDD = nonFilteredRDD.map(lambda x: checkForCandidates(x[0], x[1], x[2])).filter(lambda x: len(x[1]) > 0)